In [1]:
!wget https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/malaysia.geojson

--2021-07-02 18:51:50--  https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/malaysia.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46783 (46K) [text/plain]
Saving to: ‘malaysia.geojson.1’

malaysia.geojson.1  100%[===================>]  45.69K  --.-KB/s    in 0.001s  

2021-07-02 18:51:50 (68.6 MB/s) - ‘malaysia.geojson.1’ saved [46783/46783]



In [1]:
# !pip3 install bokeh distributed dask

In [2]:
import json
import geojson
import requests
import re
import os
import numpy as np
import dask.array as da
import dask.dataframe as dd
from shapely.geometry import mapping, shape, MultiPoint
from shapely.ops import cascaded_union

In [3]:
from dask.distributed import Client, progress
client = Client(n_workers = 16)
client

Client Scheduler: tcp://127.0.0.1:36453 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 31.36 GiB


In [4]:
with open('malaysia.geojson') as fopen:
    malaysia = json.load(fopen)

In [5]:
negeri = {k['properties']['name']:k['geometry'] for k in malaysia['features']}
negeri.keys()

dict_keys(['Federal Territory of Kuala Lumpur', 'Perlis', 'Sabah', 'Federal Territory of Putrajaya', 'Kedah', 'Sarawak', 'Penang', 'Johor', 'Kelantan', 'Melaka', 'Negeri Sembilan', 'Pahang', 'Perak', 'Selangor', 'Terengganu'])

In [7]:
# data from TindakMalaysia is more precise for these states

download = {
    'Kedah': 'https://raw.githubusercontent.com/TindakMalaysia/Kedah-Maps/master/2016/MAP/MIGRATED/result/02-Kedah-New-DM-4326.geojson',
    'Pahang': 'https://raw.githubusercontent.com/TindakMalaysia/Pahang-Maps/master/2016/MAP/MIGRATED/result/07-Pahang-New-DM-4326.geojson',
    'Johor': 'https://raw.githubusercontent.com/TindakMalaysia/Johor-Maps/master/2016/MAP/MIGRATED/result/13-Johor-New-DM-4326.geojson',
    'Perak': 'https://raw.githubusercontent.com/TindakMalaysia/Perak-Maps/master/2016/MAP/MIGRATED/result/06-Perak-New-DM-4326.geojson',
    'Labuan': 'https://raw.githubusercontent.com/TindakMalaysia/Federal-Territories-Maps/master/LABUAN/2016/MAP/MIGRATED/result/14-Labuan-New-DM-4326.geojson',
    'Terengganu': 'https://raw.githubusercontent.com/TindakMalaysia/Terengganu-Maps/master/2016/MAP/MIGRATED/result/04-Terengganu-New-DM-4326.geojson'
}

In [8]:
for k, v in download.items():
    os.system(f'wget {v}')

In [9]:
def check(df):
    points = MultiPoint(df.values)
    try:
        ls = list(points.intersection(polygon))
        return [(l.x, l.y) for l in ls]
    except Exception as e:
        return []

In [10]:
# !mkdir data

In [11]:
# meters = 1000
# earth = 6378.137
# pi = np.pi
# cos = np.cos
# m = (1 / ((2 * pi / 360) * earth)) / 1000
# default_resolution = meters * m
# default_resolution

default_resolution = 0.003
resolutions = {'Sabah': 0.005, 'Sarawak': 0.006, 'Pahang': 0.005}

In [12]:
negeris = list(negeri.keys()) + ['Labuan']
for k in negeris:
        
    print(k)
    if k in download:
        print(download[k].split('/')[-1])
        with open(download[k].split('/')[-1]) as fopen:
            data = json.load(fopen)

        state = []
        for d in data['features']:
            d = d['geometry']
            g = geojson.loads(json.dumps(d))
            polygon = shape(g)
            state.append(polygon)
        polygon = cascaded_union(state)
    else:
        s = json.dumps(negeri[k])
        g1 = geojson.loads(s)
        polygon = shape(g1)
    latmin, lonmin, latmax, lonmax = polygon.bounds 
    resolution = resolutions.get(k, default_resolution)
    lat_arange = da.arange(latmin, latmax, resolution)
    lon_arange = da.arange(lonmin, lonmax, resolution)
    x, y = da.meshgrid(lat_arange, lon_arange)
    x = da.round(x, 8)
    y = da.round(y, 8)
    x = da.reshape(x, (-1, 1))
    y = da.reshape(y, (-1, 1))
    concated = da.concatenate([x, y], axis = 1)
    df = dd.from_array(concated)
    df = df.repartition(npartitions = 100)
    check_partitions = df.map_partitions(check, meta=object)
    check_partitions = check_partitions.compute()
    inside = []
    for p in check_partitions:
        inside.extend(p)

    print(k, len(inside))
    with open(f'data/{k}-points.json', 'w') as fopen:
        json.dump(inside, fopen)

Federal Territory of Kuala Lumpur
Federal Territory of Kuala Lumpur 2139
Perlis
Perlis 7384
Sabah
Sabah 240019
Federal Territory of Putrajaya
Federal Territory of Putrajaya 404
Kedah
02-Kedah-New-DM-4326.geojson
Kedah 87668
Sarawak
Sarawak 280730
Penang
Penang 9342
Johor
13-Johor-New-DM-4326.geojson
Johor 172104
Kelantan
Kelantan 136278
Melaka
Melaka 15057
Negeri Sembilan
Negeri Sembilan 60293
Pahang
07-Pahang-New-DM-4326.geojson
Pahang 345939
Perak
06-Perak-New-DM-4326.geojson


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


Perak 189375
Selangor


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


Selangor 70600


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


Terengganu
04-Terengganu-New-DM-4326.geojson


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


Terengganu 117432
Labuan
14-Labuan-New-DM-4326.geojson
Labuan 938
